# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews


In [14]:
df = pd.DataFrame(reviews, columns=['reviews'])
df.head()

,reviews
0,✅ Trip Verified | That was supposed to be my ...
1,✅ Trip Verified | Have no fear when your BA f...
2,Not Verified | We have flown BA five times fr...
3,✅ Trip Verified | London Heathrow to Istanbul...
4,"Not Verified | Jan 30th, I booked a last-minut..."


In [ ]:
new_folder = 'data'

pwd = 'C:\\Users\\oyalu\\Documents\\Data Science\\Forage\\Britist Airways'

data_dir = os.path.join(pwd,new_folder)
data_dir

os.makedirs(data_dir)

In [49]:
df.to_csv("data/BA_reviews.csv", index=False)

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [51]:
reviews = pd.read_csv('data/BA_reviews.csv')
reviews.head()

,reviews
0,✅ Trip Verified | That was supposed to be my ...
1,✅ Trip Verified | Have no fear when your BA f...
2,Not Verified | We have flown BA five times fr...
3,✅ Trip Verified | London Heathrow to Istanbul...
4,"Not Verified | Jan 30th, I booked a last-minut..."


In [70]:
reviews_df = reviews.copy()

In [71]:
reviews_df

,reviews
0,✅ Trip Verified | That was supposed to be my ...
1,✅ Trip Verified | Have no fear when your BA f...
2,Not Verified | We have flown BA five times fr...
3,✅ Trip Verified | London Heathrow to Istanbul...
4,"Not Verified | Jan 30th, I booked a last-minut..."
...,...
995,✅ Trip Verified | London Heathrow to Madrid. I...
996,✅ Trip Verified | Gatwick to St Lucia return....
997,✅ Trip Verified | It's been many years since ...
998,✅ Trip Verified | \r\nLondon Heathrow / Pisa r...



✅ is pointless
leading spaces
| is not supposed to be there
trip verification can be in separate column
\r\n in some entries

remove ✅
split by |
remove leading spaces
remove \r\n

In [74]:
reviews_df = pd.DataFrame(reviews_df.reviews.str.replace('✅',''))

In [75]:
reviews_df

,reviews
0,Trip Verified | That was supposed to be my f...
1,Trip Verified | Have no fear when your BA fl...
2,Not Verified | We have flown BA five times fr...
3,Trip Verified | London Heathrow to Istanbul ...
4,"Not Verified | Jan 30th, I booked a last-minut..."
...,...
995,Trip Verified | London Heathrow to Madrid. I ...
996,Trip Verified | Gatwick to St Lucia return. ...
997,Trip Verified | It's been many years since I...
998,Trip Verified | \r\nLondon Heathrow / Pisa re...


In [96]:
reviews_df

,reviews
0,Trip Verified | That was supposed to be my f...
1,Trip Verified | Have no fear when your BA fl...
2,Not Verified | We have flown BA five times fr...
3,Trip Verified | London Heathrow to Istanbul ...
4,"Not Verified | Jan 30th, I booked a last-minut..."
...,...
995,Trip Verified | London Heathrow to Madrid. I ...
996,Trip Verified | Gatwick to St Lucia return. ...
997,Trip Verified | It's been many years since I...
998,Trip Verified | \r\nLondon Heathrow / Pisa re...


In [109]:
reviews_df[['verification','review']] = reviews_df['reviews'].str.split('|', expand=True)

In [110]:
reviews_df

,reviews,verification,review
0,Trip Verified | That was supposed to be my f...,Trip Verified,That was supposed to be my flight but it was...
1,Trip Verified | Have no fear when your BA fl...,Trip Verified,Have no fear when your BA flight is operated...
2,Not Verified | We have flown BA five times fr...,Not Verified,We have flown BA five times from western USA...
3,Trip Verified | London Heathrow to Istanbul ...,Trip Verified,London Heathrow to Istanbul at the start of ...
4,"Not Verified | Jan 30th, I booked a last-minut...",Not Verified,"Jan 30th, I booked a last-minute, transatlant..."
...,...,...,...
995,Trip Verified | London Heathrow to Madrid. I ...,Trip Verified,London Heathrow to Madrid. I travel from LHR ...
996,Trip Verified | Gatwick to St Lucia return. ...,Trip Verified,Gatwick to St Lucia return. Recently returne...
997,Trip Verified | It's been many years since I...,Trip Verified,It's been many years since I flew long haul ...
998,Trip Verified | \r\nLondon Heathrow / Pisa re...,Trip Verified,\r\nLondon Heathrow / Pisa return. My wife an...
